In [ ]:
import geopandas as gpd
import pandas as pd
import structlog
import yaml
from shapely import from_wkt


def create_lookup(essential_amenities: dict[str, list[str]]) -> dict[str, str]:
    result = {}
    for cat, osm_list in essential_amenities.items():
        for i in osm_list:
            result[i] = cat
    return result


with open("../data/essential_amenities.yaml") as fp:
    essential_amenities = yaml.safe_load(fp)

logger = structlog.get_logger()

lookup = create_lookup(essential_amenities)

In [ ]:
CITY = "budapest"

In [3]:
amenities = gpd.read_file(
    f"../output/{CITY}/amenities/amenities_filtered.wkt.csv", engine="pyogrio"
)
amenities.rename({"category": "osm_category"}, axis=1, inplace=True)
amenities["category"] = amenities["osm_category"].map(lookup)
amenities.dropna(subset=["category"], inplace=True)
amenities.drop(["osm_type", "amenity_type", "amenity_subtype"], axis=1, inplace=True)
amenities["geometry"] = amenities["geometry"].apply(from_wkt)
amenities = gpd.GeoDataFrame(amenities, geometry="geometry", crs=4326)
amenities.head(3)

,osm_id,osm_category,name,geometry_type,geometry,category
8,30761500,amenity:pharmacy,Valentin Gyógyszertár,Point,POINT (17.74846 46.88412),drugstores
9,30761501,amenity:restaurant,Fék Étterem,Point,POINT (17.74914 46.88292),restaurants
12,31929662,amenity:restaurant,Kisherceg vendéglő,Point,POINT (19.22771 47.52502),restaurants


In [4]:
isochrones = gpd.read_file(f"../output/{CITY}/isochrones.geojson", engine="pyogrio")
isochrones.head(3)

,stop_id,costing,range,geometry
0,044369,walk,5,"POLYGON ((18.9428 47.17896, 18.9418 47.17947, ..."
1,044369,walk,10,"POLYGON ((18.9428 47.18272, 18.9408 47.18326, ..."
2,044369,walk,15,"POLYGON ((18.9408 47.18702, 18.9398 47.18529, ..."


In [ ]:
isochrones.query("stop_id=='007884'").query("range == 5 & costing=='walk'").to_file(
    "../007884.geojson"
)

In [ ]:
result = pd.DataFrame()
for i in isochrones[["costing", "range"]].drop_duplicates().itertuples():
    ac = (
        isochrones.query(f"range == {i.range} & costing=='{i.costing}'")
        .sjoin(amenities)
        .groupby(["stop_id", "category"])["osm_id"]
        .count()
        .reset_index()
    )
    ac.rename({"osm_id": "count"}, axis=1, inplace=True)

    ac = (
        pd.pivot_table(ac, index=["stop_id"], columns=["category"], values="count")
        .fillna(0)
        .map(int)
        .reset_index()
    )

    ac["costing"] = i.costing
    ac["range"] = i.range
    result = pd.concat([result, ac])

result.to_csv(f"../output/{CITY}/amenity_counts_in_accessibility.csv", index=False)

      stop_id                 category  count
0      002133                    parks      2
1      002133                  schools      1
2      002133                 services      1
3      002138    cultural_institutions      1
4      002138               drugstores      4
...       ...                      ...    ...
30168  LMXKPN                    parks      4
30169  LMXKPN  religious_organizations      3
30170  LMXKPN              restaurants     41
30171  LMXKPN                  schools      4
30172  LMXKPN                 services     12

[30173 rows x 3 columns]
category stop_id  cultural_institutions  drugstores  groceries  healthcare  \
0         002133                      0           0          0           0   
1         002138                      1           4         10           1   
2         003002                      1           1          0           0   
3         003103                      0           0          4           0   
4         003105                

In [33]:
sgfw = gpd.read_file(f"../output/{CITY}/stop_geometries_from_walk.geojson")

In [ ]:
temp = sgfw.sjoin(amenities)
temp = temp.groupby(["stop_id", "category"])["osm_id"].count().reset_index()
temp.rename({"osm_id": "count"}, axis=1, inplace=True)
temp = (
    pd.pivot_table(temp, index=["stop_id"], columns=["category"], values="count")
    .fillna(0)
    .map(int)
    .reset_index()
)
temp["costing"] = "public_transport"
# temp["range"] = "10+5"
temp["range"] = pd.NA
temp.to_csv(
    f"../output/{CITY}/amenity_counts_in_public_transport_accessibility.csv",
    index=False,
)